In [ ]:
#Murat Gün Search Engine Project
def get_page(url):
  try:
    import urllib.request
    page = urllib.request.urlopen(url).read()
    page = page.decode("utf-8")
    return page
  except:
    return ""


def get_next_target(page):
    start_link = page.find('<a href=')
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote+1)
    url = page[start_quote + 1:end_quote]
    return url, end_quote


def get_all_links(page):
    links = []
    while True:
      url, endpos = get_next_target(page)
      if url:
        links.append(url)
        page = page[endpos:]
      else:
        break
    return links


def union(p,q):
    for e in q:
        if e not in p:
            p.append(e)


def add_toIndex(index, keyword, url):
    if keyword in index:
        index[keyword].append(url)
    else:
        index[keyword] = [url]

def getclearpage(content):
  title = content[content.find("<title>")+7:content.find("</title>")]
  body = content[content.find("<body>")+6:content.find("</body>")]
  while body.find(">") != -1:
    start =  body.find("<")
    end =  body.find(">")
    body = body[:start] + body[end+1:]
  return title + body


def addPageToIndex(index, url, content):
  content = getclearpage(content)
  words = content.split()
  for word in words:
    add_toIndex(index, word, url)

def crawlWeb(seed):
  tocrawl = [seed]
  crawled = []
  index = {}
  graph = {}
  while tocrawl:
    page = tocrawl.pop()
    if page not in crawled:
      content = get_page(page)
      addPageToIndex(index, page, content)
      outlinks = get_all_links(content)
      graph[page] = outlinks
      union(tocrawl, get_all_links(get_page(page)))
      crawled.append(page)
  return index, graph

def originallookup(index, keyword):
    if keyword in index:
        return index[keyword]
    else:
        return None

def computeRanks(graph):
    d =  0.8 # damping factor
    N = len(graph)
    numloops = 10  # effects the accuracy
    ranks = {}
    for page in graph:
        ranks[page] = 1/N
    for i in range(0, numloops):
        newranks = {}
        for page in graph:
            newrank = (1-d)/N
            for node in graph:
                if page in graph[node]:
                    newrank = newrank + d*(ranks[node] / len(graph[node]))
            newranks[page] = newrank
        ranks = newranks
    return newranks



# This graph for a given a seed page gives a dictionary in which elements of it are links that are reached from the seed page or
# links that are reached from the reached ones. In the graph, value of a link (since it is a dictionary, I use that terminology)
# is a list of links that are reached by the given link. "Reaching" in that context is having the link in the source code.


#1b

def urlsAsStrings(list):
    string = ""
    for i in list:
        string = string + i + ", "
    finalString = string[:-2]
    return finalString

seed = "https://ayaktayolcukalmasin.com.tr/ana_sayfa.html"
index1,graph1 = crawlWeb(seed)
print("This graph has " + str(len(graph1)) + " elements. These are:")
counter = 1
for elements in graph1:
    String = urlsAsStrings(graph1[elements])
    print("\t" + str(counter) + ".  " + elements + ": " + String)
    counter = counter + 1


#1c

def computeRanks(graph):
    d =  0.8 # damping factor
    N = len(graph)
    numloops = 10  # effects the accuracy
    ranks = {}
    for page in graph:
        ranks[page] = 1/N
    for i in range(0, numloops):
        newranks = {}
        for page in graph:
            newrank = (1-d)/N
            for node in graph:
                if page in graph[node]:
                    #print(len(graph[node]))
                    #print(graph[node])
                    newrank = newrank + d*(ranks[node] / len(graph[node]))
            newranks[page] = newrank
        ranks = newranks
    return newranks

computeRanks(graph1)
for page in graph1:
    print("The rank of the page " + page + ":  " + str(computeRanks(graph1)[page]))

#1d

def functionForDictOrder(dict):                 #I defined that functions which orders the elements in descending values
    newDict = {}                                #f.e {"link1":0.2, "link2": 0.1, "link3":0.4} gets {'link3': 0.4, 'link1': 0.2, 'link2': 0.1}.
    for i in dict:
        newDict[dict[i]] = i
    listofValues = sorted(newDict, reverse=True)
    dictOrdered = {}
    for i in listofValues:
        dictOrdered[newDict[i]] = i
    return dictOrdered
def rankedLookup(index, keyword , graph):
    dict = {}
    x = index[keyword]
    for i in x:
        a,b = i,(computeRanks(graph)[i])
        dict[a] = b
    return functionForDictOrder(dict)



#1e

def rankedLookup(index, keyword , graph, rankmethod): #Updated version of rankedLookup where it gets also "rankmethod" argument
    dict = {}
    x = index[keyword]
    for i in x:
        a,b = i,(rankmethod(graph)[i])
        dict[a] = b
    return functionForDictOrder(dict)


def lookup (index , keyword, graph=None, rankmethod=None):
    if graph==None and rankmethod==None:
        return originallookup(index, keyword) #I renamed the first version of "lookup" into "originallookup"
    elif graph==None or rankmethod==None:
        print("This procedure takes 4 inputs these are:" + "\n\t" + "1-An index" + "\n\t"
              "2-A key" + "\n\t" + "3-A graph" + "\n\t" + "4-A computing procedure" + "\n\t\trespectively." +
              "\nYou have 2 option to use this lookup procedure: with or without keyword.\n" +
              "\t-If you intend to use it without page rank be sure you have given only two inputs, index and key, respectively."
              + "\n\t-If you intend to use it with page rank be sure you have given all four input in the given order." +
              "\nINVALID INPUT COMBINATION: Please check the inputs." )
        exit()
    else:
        return rankedLookup(index, keyword, graph, rankmethod)

see = lookup(index1, "in", graph1, computeRanks)
print("\t\nRanking from more popular to less for the key:")
for e in see:
  print(e)

This graph has 6 elements. These are:
	1.  https://ayaktayolcukalmasin.com.tr/ana_sayfa.html: https://ayaktayolcukalmasin.com.tr/ankara.html, https://ayaktayolcukalmasin.com.tr/konya.html, https://ayaktayolcukalmasin.com.tr/istanbul.html, https://ayaktayolcukalmasin.com.tr/oktayrecommends.html, https://ayaktayolcukalmasin.com.tr/seymarecommends.html
	2.  https://ayaktayolcukalmasin.com.tr/seymarecommends.html: https://ayaktayolcukalmasin.com.tr/oktayrecommends.html, https://ayaktayolcukalmasin.com.tr/konya.html
	3.  https://ayaktayolcukalmasin.com.tr/oktayrecommends.html: https://ayaktayolcukalmasin.com.tr/istanbul.html
	4.  https://ayaktayolcukalmasin.com.tr/istanbul.html: 
	5.  https://ayaktayolcukalmasin.com.tr/konya.html: https://ayaktayolcukalmasin.com.tr/seymarecommends.html
	6.  https://ayaktayolcukalmasin.com.tr/ankara.html: 
The rank of the page https://ayaktayolcukalmasin.com.tr/ana_sayfa.html:  0.033333333333333326
The rank of the page https://ayaktayolcukalmasin.com.tr/seym